In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib

# pip install graphviz
# conda install python-graphviz

In [ ]:
matplotlib.rcParams.update({'font.size': 18,
                            'lines.linewidth' : 3,
                           'figure.figsize' : [15, 5],
                           'lines.markersize': 10})
pd.options.mode.chained_assignment = None


# Intro to Machine Learning

What does it mean to learn?  Learning is a process where we take a series of observations and draw conclusions based on past experiences. For example, we can learn to recognize patterns in experiential data such as when I take the later bus, I'm late to work.  Machine Learning is when we teach a computer to do the same thing, namely find patterns in data.  The idea is that humans are really great at finding patterns, but relatively slow at looking through large amounts of data.  Computers need to be trained to find the patterns, but they can process data of the sort of we have (csv files, images, etc) incredibly fast.

If we want to leverage machine learning, we need to teach computers to recognize patterns and leverage that ability to solve real world patterns.  Lets start with a really simple example.

Say we have one dimensional data given by a single feature $X$ and a corresponding set of labels $y$.  We want to model this data, so we will create a relationship \begin{equation} f(X) \approx y .\end{equation} This function $f$ will represent our model.  We will generate the data here by randomly choosing an exponent for a trend and adding some random noise.  Lets create the data and see what this looks like.

In [ ]:
X = np.linspace(0, 1, 100)
print("X: size {}".format(len(X)))
print(X)
# exp = np.random.choice([2, 3])
exp = 2
y = X**exp + np.random.randn(X.shape[0])/10
print("")
print("y: size {}".format(len(y)))
print(y)

plt.plot(X, y, '.');
plt.xlabel('x')
plt.ylabel('y');

We will now generate the predictive relationship by using one of the simplest possible methods, fitting a line to the data $$y=mx+c$$

In [ ]:
from ipywidgets import interact, FloatSlider

def plotBestLine(m,c):
    plt.plot(X, y, '.', label='data');
    plt.plot(X, m*X + c, label = 'Model: {}x+{}'.format(m,c))
    plt.legend();

slider_m = FloatSlider(value=1, min=0, max=2, step=0.02, description='Gradient (m)')    
slider_c = FloatSlider(value=0, min=-0.5, max=0.5, step=0.02, description='Intercept (c)')    
interact(plotBestLine, m=slider_m, c=slider_c);

In [ ]:
p = np.polyfit(X, y, 1)
z = np.poly1d(p)
plt.plot(X, y, '.')
plt.plot(X, z(X), label=r"Model: ${:.2f}x + {:.2f}$".format(*p))
plt.plot(X, X**exp, label=r'Truth: $x^{}$'.format(exp))
plt.legend();

We now have a model for this data, learned by the computer, namely given an $X$ value (or a bunch of values), we can predict the output.  This example opens up many questions:

1. How good is the model?
2. Can we add flexibility to the model?
3. Is the model generalizable?
4. What does this model teach us about the data?

Lets start with question 4, which in many ways is the most important question.  For this simple model we can see that the $y$ vector of labels has a positive correlation with the features $X$.

## Why do we do Machine Learning?
Normally the goal of machine learning is two-fold

1. To understand the data we already have
2. Use this understanding to make predictions about unlabeled data

Machine Learning falls into two classes, 
- **supervised** learning : *learn a predictive relationship between **features** of our data and some sort of output **target** label* 
- **unsupervised** learning : *find trends in our features without using any target labels*

A human example of supervised learning would be borrowing books from a library on mathematics and geography. By reading different books belonging to each topic, we learn what symbols, images, and words are associated with math, and which are associated with geography. 

A similar unsupervised task would be to borrow many books without knowing their subject. We can see some books contain similar images (maps) and some books contain similar symbols (e.g. the Greek letters $\Sigma$ and $\pi$). We say the books containing maps are similar and that they are different from the books containing Greek letters. Crucially, _we do not know what the books are about, only that they are similar or different_.

## Supervised Machine learning
Formally, the supervised machine problem can be stated as:
- given a matrix $X$,
of dimensions $n \times p$
- create a predictive relationship (or function) $f(X)$ where \begin{equation} f(X) \approx y \end{equation} 
    - $y$ is a vector of dimension $n$ called the **labels**.
    - $X$ is referred to as the **feature matrix** 

In [ ]:
from sklearn.datasets import fetch_california_housing
dt_cali = fetch_california_housing()
df_cali = pd.DataFrame(dt_cali.data, columns=dt_cali.feature_names)
df_target = [1 if ii < 2 else 0  for ii in dt_cali['target'] ]

In [ ]:
numRows = 20
df_sup = df_cali.head(numRows)
df_sup['-'] = '--------'
df_sup['MedianHouseValue'] = dt_cali['target'][:numRows]
# df_sup['AffordableHouse'] = df_target[:numRows]
print('Supervised Learning Dataset:')
df_sup

The general goal of supervised learning is to then apply this model to unlabeled data where can build a feature matrix representative of the original.  This allows us to make predictions! 

In [ ]:
df_sup = df_cali.head(numRows+5)[-5:]
df_sup['-'] = '--------'
df_sup['MedianHouseValue'] = '?'
# df_sup['AffordableHouse'] = '?'
df_sup

In [ ]:
print('Unsupervised Learning Dataset:')
df_cali.head(numRows)

## Machine Learning Difficulties
Machine learning is just a tool and is not a 'one-fits-all' solution.

Models can be heavily biased and thus not flexible enough to handle generalization.  Let us plot our original function over a larger range and use the model from before.

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2)
X = np.linspace(0, 1, 100)
y = X**exp + np.random.randn(X.shape[0])/10
p = np.polyfit(X, y, 1)
z = np.poly1d(p)
ax[0].plot(X, y, '.')
ax[0].plot(X, z(X), '-', linewidth=4, label= "Model: ${:.2f}x + {:.2f}$".format(*p))
ax[0].plot(X, X**exp,'-', label=r'Truth: $x^{}$'.format(exp))
ax[0].legend();
ax[0].set_title('Original Model');

X = np.linspace(0, 2, 100)
y = X**exp + np.random.randn(X.shape[0])/10
ax[1].plot(X, y, '.')
ax[1].plot(X, z(X), 'g', linewidth=4, label= "Model: ${:.2f}x + {:.2f}$".format(*p))
ax[1].plot(X, X**exp,'-', color='r', label=r'Truth: $x^{}$'.format(exp))
ax[1].legend();
ax[1].set_title('Extending $x$-axis');

The model works fairly well for the range over which initially considered our data, but we can see it will not generalize well to features outside the of the range we considered.  This is a general problem; we should be careful that our training data contains a well sampled distribution over which we expect to make predictions (or we have some prior knowledge that tells us we should be able to extrapolate beyond the domain of our training data).  Machine learning finds patterns in data that it's already seen, and it can't always make good predictions on data it hasn't. 

Lets try to fix this by adding more parameters to the model.

In [ ]:
p = np.polyfit(X, y, 15)
z = np.poly1d(p)
plt.figure(figsize=[14, 6])
plt.plot(X, z(X), 'g', label=r"${:.2f}x^{{15}} + {:.2f}x^{{14}} + ... + {:.2f}$".format(*p[[0, 1, -1]]))
plt.plot(X, y,'.', label=r'$x^{}$'.format(exp))
plt.legend();

Wow looks like a perfect fit!  Maybe too good?  It looks like the model is fitting little wiggles in the data which we know are not real (the actual data is derived from a simple exponent).  Lets try to generalize again.

In [ ]:
X = np.linspace(0, 2.5, 100)
y = X**exp + np.random.randn(X.shape[0])/10
plt.plot(X, z(X), 'g', linewidth=4, label=r"model")
plt.plot(X, y,'.', label=r'$x^{}$'.format(exp))
plt.legend();

Wow again!  That is pretty bad.  This is an example of overfitting, where we have allowed the model too much flexibility and it has fit the noise in the data which is not generalizable.

We will learn more how to combat these issues, but the point is that we need to be careful when choose the model we want to use and the **hyperparameters** (in this case order of the polynomial) for the model.

# Intro to Scikit-learn

`Scikit-learn` is the most popular Python package for machine learning. It has a plethora of machine learning models and a nice and intuitive interface. It makes creating complicated machine learning workflows very easy. 

## Machine Learning models as classes

`Scikit-learn` relies heavily on object-oriented programming principles. It implements machine learning algorithms as classes and users create objects from these "recipes". For example, `LinearRegression` is a class representing the Linear Regression model. To create a `lr` object, we simply create an instance of the class. In Python, the convention is that class names use CamelCase, the first letter of each word is capitalized. `Scikit-learn` adopts the convention, making it easy to distinguish what is a class.

In [ ]:
from sklearn.linear_model import LinearRegression

linear = LinearRegression(fit_intercept=False)

In the above code, we set `fit_intercept=False`. Here, `fit_intercept` is a **hyperparameter** of the Linear Regression model. Hyperparameters are model parameters that govern the learning process. In terms of hierarchy, they reside "above" the regular model parameters. They control what values the model parameters are equal to after undergoing training. They can be easily identified as they are the parameters that are set _prior_ to learning. In `scikit-learn`, hyperparameters are set when creating an instance of the class. The default values that `scikit-learn` uses are *usually* a good set of initial values but this is not always the case. It is important to understand the hyperparameters available and how they affect model performance.

`Scikit-learn` refers to machine learning algorithms as **estimators**. There are three different types of estimators: 
1. Classifiers, 
1. Regressors, and 
1. Transformers. 

Programmatically, `scikit-learn` has a base class called `BaseEstimator` that all estimators inherit. The models inherit an additional class, either `RegressorMixin`, `ClassifierMixin`, and `TransformerMixin`. The inheritance of the second class determines what type of estimator the model represents. We'll divide the estimators into two groups based up on their interface. These two groups are **predictors** and **transformers**.

Full information is available in the [documentation](https://scikit-learn.org/stable/index.html)

## Predictors: Classifiers and Regressors

As the name suggests, predictors are models that make predictions. There are two main methods.

* `fit(X, y)`: trains/fit the object to the feature matrix $X$ and label vector $y$.
* `predict(X)`: makes predictions on the passed data set $X$.

In [ ]:
from sklearn.linear_model import LinearRegression

# create model and train/fit
model = LinearRegression()
model.fit(df_cali, df_target)

# predict label values on X
y_pred = model.predict(df_cali)

print("Prediction : {}".format(y_pred))
print("     Shape of the prediction array: {}".format(y_pred.shape))
print("     Shape of the training set: {}".format(X.shape))

Note, the output of `predict(X)` is a NumPy array of one dimension. The array has the same size as the number of rows of the data that was passed to the `predict` method. 

Since we are using linear regression and our data has eight features, our model is

$$ y(X) = \beta_1 x_1 + \beta_2 x_2 + \beta_3 x_3 + \beta_4 x_4 + \beta_5 x_5 + \beta_6 x_6 + \beta_7 x_7 + \beta_8 x_8 + \beta_0. $$

The coefficients are stored in the fitted model as an object's attribute. `Scikit-learn` adopts a convention where all attributes that are determined/calculated _after_ fitting end in an underscore. The model coefficients and intercept are retrieved using the `coefs_` and the `intercept_` attributes, respectively.

In [ ]:
print("β_0: {}".format(model.intercept_))
for i in range(8):
    print("β_{}: {}".format(i+1, model.coef_[i]))

If we wanted to know how well the model performs making predictions with a data set, we can use the `score(X, y)` method. It works by

1. Internally running `predict(X)` to produce predicted values.
1. Using the predicted values to evaluate the model compared to the true label values that were passed to the method.

The evaluation equation varies depending if the model is a regressor or classifier. For regression, it is the $R^2$ value while for classification, it is accuracy.

In [ ]:
print("R^2: {:g}".format(model.score(df_cali, df_target)))

We used a rather simple model, linear regression. What if we wanted to use a more complicated model? All we need to do is an easy substitution; there is minimum code rewrite as the models have the same interface. Of course, different models have different hyperparameters so we need to be careful when swapping out algorithms. Let's use a more complicated model and train it.

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

# create model and train/fit
model = GradientBoostingRegressor()
model.fit(df_cali, df_target)

# predict label values on X
y_pred = model.predict(df_cali)

print(y_pred)
print("R^2: {:g}".format(model.score(df_cali, df_target)))

## Transformers

Transformers are models that process and transform a data set. These transformers are very useful because rarely is our data in a form to feed directly to a machine learning model for both training and predicting. All transformers have the same interface:

* `fit(X)`: trains/fits the object to the feature matrix $X$.
* `transform(X)`: applies the transformation on $X$ using any parameters learned
* `fit_transform(X)`: applies both `fit(X)` and then `transform(X)`.

Let's demonstrate transformers with `StandardScaler`, which scales each feature to have zero mean and unit variance. The transformed feature $x'_i$ is equal to
\begin{equation} x'_i = \frac{x_i - \mu_i}{\sigma_i} \end{equation}

In [ ]:
from sklearn.preprocessing import StandardScaler

# create and fit scaler
scaler = StandardScaler()
scaler.fit(df_cali)

# scale data set
Xt = scaler.transform(df_cali)

# create data frame with results
stats = np.vstack((df_cali.mean(axis=0), df_cali.var(axis=0), Xt.mean(axis=0), Xt.var(axis=0))).T
feature_names = df_cali.columns
columns = ['unscaled mean', 'unscaled variance', 'scaled mean', 'scaled variance']

pd.DataFrame(stats, index=feature_names, columns=columns)

The data frame shows how our features have wildly different scales; the average population is over 1000 but the average room is slightly over 5. Now, our features each have zero mean and a variance of one.

# Supervised Machine Learning for a Classification Problem

## Dataset - Titanic Likelihood of Survival 
For this notebook, we will use the [Titanic Likelihood of Survival](https://www.kaggle.com/c/titanic/data) data. The data set contains the likelihood of survival of a person on board of the titanic.

In [ ]:
df_titanic = pd.read_csv('./00_data/titanic.csv')

In [ ]:
print(df_titanic.shape)
df_titanic.head()

**Data dictionary**

| Variable	| Definition|	Key |
|---|---|---|
|survived |	Survival |	0 = No, 1 = Yes|
|pclass |	Ticket class |	1 = 1st, 2 = 2nd, 3 = 3rd|
|sex 	|Sex 	||
|Age |	Age in years 	||
|sibsp 	|# of siblings / spouses aboard the Titanic 	||
|parch 	|# of parents / children aboard the Titanic 	||
|ticket 	|Ticket number 	||
|fare 	|Passenger fare 	||
|cabin |	Cabin number 	||
|embarked |	Port of Embarkation |	C = Cherbourg, Q = Queenstown, S = Southampton|

**Variable Notes**
- *pclass* : A proxy for socio-economic status
    - 1st = Upper
    - 2nd = Middle
    - 3rd = Lower
- *age* : Age is fractional if less than 1. If the age is estimated, is it in the form of xx.5
- *sibsp* : The dataset defines family relations in this way...
    - Sibling = brother, sister, stepbrother, stepsister
    - Spouse = husband, wife (mistresses and fiancés were ignored)
- *parch* : The dataset defines family relations in this way...
    - Parent = mother, father
    - Child = daughter, son, stepdaughter, stepson
    - Some children travelled only with a nanny, therefore parch=0 for them.

## Mathematics of supervised learning

For supervised ML, our model receives a vector of **features**, $X$, and maps it to some predicted label, $y$. In order to train our model, we will need many **observations** (i.e. measurements) and their associated labels. We can assemble these observations into a matrix.
$$ f(X_{ij}) \approx y_i $$

In [ ]:
df_titanic.head()

In the above dataframe, each column is a feature (i.e. a variable) and each row is an observation (i.e. a measurement). Said another way, things like **pclass**, **sex** and **age** are features. The column **survived** is the target labels.

## Metrics for Classification Problems
In classification, we predict categorical labels. In regression, we predict quantitative/numerical labels. The critical difference is that we can't take a difference between the predicted and actual category in classification, while we can take a difference between the predicted and actual numerical values in regression. Because of these differences between regression and classification, we use different metrics to evaluate machine learning models trained for classification.

We are trying to determine the model $f$ that can best describes the relationship

$$ y_j = f(X_j). $$

For classification, $y_j$ can only take a finite set of values. If there are only two such values, we are dealing with **binary** classification. Examples of binary classification are predicting whether it will rain or not and whether someone will default on their loan. If we have more than two classes, we have a **multiclass** problem. For example, image classification is usually multiclass as we are trying to identify an image among a set of values, e.g., a person, a road sign, a car, etc.

### Accuracy

A natural choice for a metric for classification is accuracy. Accuracy is equal to the number of observations you correctly classified over all observations. For example, if your model properly identified 77 out of 100 images, you have an accuracy of 77%. Accuracy is an easy metric to both understand and calculate. Mathematically, it is simply

$$ \frac{\text{number of correct observations}}{\text{number of observations}}.$$

However, accuracy may not always be a good metric. 

Consider the case of disease detection where only 10% of the observations have the disease. A "stupid" classifier that always predicts the majority class. For example, consider 100 observations with 10 rows as being infected. If the model predicts all as "No disease",

\begin{align}
\text{True Label} &: \left[0,0,0,0,0,0,0,0,0,0,0,...,0,1,1,1,1,1,1,1,1,1,1\right]\\
\text{Predicted } &: \left[0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0\right] \\
\\
\frac{\text{number of correct observations}}{\text{number of observations}}
&= \frac{90}{100} = 90\%
\end{align}

*Realistically, this model has learning "nothing"* as it fails to identify any person with the disease. 

We need a metric that will tell us how well our model performs for a particular class.

### Precision, Recall, f1-Score and Confustion Matrix
####  Precision & Recall
For the example of disease detection, we are more interested in determining our model's performance with regards to the class representing having the disease. Let's call this class  **positive** and not having the disease as **negative**. 

We want to know what fraction of all positive predictions were correct and what fraction of positive observations did we identify. The two metrics that describe these values are **precision** and **recall**. 
- Precision is the fraction of true positives over all positive predictions. It is a measure of how "precise" our model was with regards to labeling observations as positive. 
- Recall is the fraction of true positives over all positive observations. It is a measure of our model's ability to "catch" and properly label observations that are positive.

To summarize these 2 metrics, we construct a **confusion matrix**. This is a table summarizing the performance of the model by enumerating true and false positives and the true and false negatives.


|             *       | Positive Observation (1)   | Negative Observation (0)   |
|---------------------|:------------------------:|:-----------------------:|
| Positive Prediction (1) |     True Positive (TP)   | False Positive (FP)     |
| Negative Prediction (0) | False Negative (FN)      |     True Negative (TN)  |



Given the definitions used earlier, the equation for precision and recall are

$$ \text{precision} = \frac{\text{TP}}{TP + FP}$$
and
$$ \text{recall} = \frac{\text{TP}}{TP + FN}. $$

Note, the difference between the metrics is their denominator. 

In our disease detection example, if our data had 10 observations as true disease, 

\begin{align}
\text{True Label} &: \left[1,1,1,1,1,1,1,1,0,0,0,0,0,0,1,1\right] \\
\text{Predicted}  & :\left[1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0\right] \\
\\
\text{12 observations were marked as infected} & \\
\text{precision} & = \frac{\text{TP}}{TP + FP} = \frac{8}{8+4} = 0.667\% \\
\\
\text{8 were correctly identified or "recalled"} &\\
\text{recall} &= \frac{\text{TP}}{TP + FN} = \frac{8}{8+2} = 0.80\%
\end{align}

If we had used the previous "simple" model that predicts the all as 0 class, the recall would be 0 and our precision would be undefined.

What is more important, precision or recall? The answer depends on the specifics of the problem. Having a model that prioritizes less false positives will have a higher precision and a model that aims to reduce the number of false negatives will have a higher recall. You must decide whether your model should prioritize reducing false positives or false negatives. It is often helpful to consider the cost, whether financial, societal, etc., of your model making false positives and false negatives.

**Questions**
* For disease detection, is it better to have a higher precision or recall?
****
* Does our answer change if we need to have diagnosed patients undergo invasive and risky procedures?
****

#### f1-Score
In addition to precision and recall, there is that f1-score which is the harmonic mean of precision and recall. It is a nice metric to use when we don't have a preference over precision and recall. 
$$ \text{f1-score} = 2\times\frac{\text{precision}\times\text{recall}}{\text{precision}+\text{recall}} $$

#### Classification Report
We can easily calculate all these metrics using the `sklearn.metrics` module and to summarise all metrics, we will use the function `metrics.classification_report`. This will calculate the metrics for both scenarios of what class is considered positive or negative.

In [ ]:
import numpy as np
from sklearn import metrics

# generate our results
y_pred = np.zeros(100, dtype=np.int32)
y_pred[:12] = 1
y = np.zeros(100)
y[:8] = 1
y[-2:] = 1

print('True Labels : ')
print([int(ii) for ii in y])
print('\nPredicted Labels :')
print([int(ii) for ii in y_pred])
print("\nPrecision: {:g}".format(metrics.precision_score(y, y_pred)))
print("Recall: {:g}".format(metrics.recall_score(y, y_pred)))
print("Classification Report:")
print(metrics.classification_report(y, y_pred))

## Probabilistic Classification Models
Some classification models do not directly predict a class for an observation but instead reports a probability. For example, it might predict that there's a 75% chance the observation is positive. For the preceding example, should we assign a positive or negative label? The natural choice is to assign the observation as positive since the predicted probability is greater than 50%. However, we don't have to stick to 50%; we can adjust our **threshold** and only classify observations as positive if our models predicts a greater than 90% probability. By increasing the threshold, we will make our model only make positive predictions when it is very certain and confident. Conversely, if we lower our threshold, our model will more liberally assign positive labels. Adjusting threshold affects the models precision and recall. 

As we started to see earlier, there is tradeoff between precision and recall that becomes more apparent with probabilistic models. Let's explore and visualize the tradeoff between precision and recall. We'll generate some data 

In [ ]:
# generate data
np.random.seed(0)
y_proba = np.linspace(0, 1, 1000)
y_pred = (y_proba > 0.5).astype(np.int32)
y = np.random.binomial(1, y_proba)

print("accuracy: {}".format(metrics.accuracy_score(y, y_pred)))

In [ ]:
precision, recall, threshold = metrics.precision_recall_curve(y, y_proba)
f1_score = 2*precision*recall/(precision + recall)
threshold = np.hstack((0, threshold))

plt.plot(threshold, precision)
plt.plot(threshold, recall)
plt.plot(threshold, f1_score)
plt.xlabel('threshold')
plt.legend(['precision', 'recall', '$F_1$']);

In the above figure, we see how increasing the threshold led to higher precision but lower recall. The threshold that yielded the largest $F_1$ score was about 0.36. Any probabilistic model can achieve any arbitrary level of precision and recall by adjusting the threshold. As such, when comparing the performance of probabilistic classifiers, we need a single metric that is not dependent on threshold.

### Area under the curve
The precision-recall curve illustrates the tradeoff for a particular classifier. While there will always be a tradeoff between these two metrics, ideally the tradeoff should not be severe. In other words, the model should not sacrifice a large amount of precision to slightly improve recall. We can visualize the degree of the tradeoff by plotting what is known as a precision-recall curve.

In [ ]:
plt.plot(recall, precision)
plt.xlabel('recall')
plt.ylabel('precision')
plt.xlim([0, 1])
plt.ylim([0, 1]);

We want a model that has less tradeoff between precision and recall, resulting in a curve with less of a drop with increasing recall. Geometrically, it is better to have a model with a larger area under the curve, **AUC**, of its precision-recall plot. In `scikit-learn`, the AUC can be calculated using the `metrics.auc` function. In addition to **AUC**, there is the **ROC-AUC** metric which is based on the receiver-operator curve (ROC). The ROC plots the true positive rate against the false negative rate.

In [ ]:
print("Precision-Recall AUC: {}".format(metrics.auc(recall, precision)))
print("Receiver-Operator AUC: {}".format(metrics.roc_auc_score(y, y_proba)))

In the example, the resulting model had similar values for AUC and ROC. In general, if your data is imbalanced (more observation of the negative class) or if you care more about false positives you should rely on AUC of the precision-recall curve. Note, the number of true negatives are not factored in calculating either precision or recall.

# Summary
1. What is ML
1. How to use `scikit-learn`
    - **predictors**: `fit(X)`, `predict(X)`
    - **transformers**: `fit(X)`, `transform(X)`,`fit_transform(X)`
1. What metrics to use for Supervised classification learning    
    - Accuracy,
    - Precision, Recall, f1-score
    - Confusion Matrix/Classification Report
    - Area under curve (AUC)